#Bibliotecas

##Instalação

In [1]:
# Caso precise intalar as bibliotecas
# !pip install qiskit
# !pip install qiskit-aer
# !pip install qiskit-ibm-runtime
# !pip install qiskit-ibmq-provider
# !pip install pylatexenc



##Importação

In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService

from qiskit import QuantumCircuit, Aer, transpile, execute
from qiskit.compiler import transpile
from qiskit_ibm_provider import IBMProvider
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.result import Result

from numpy import pi, sin, cos, array, dot, outer, transpose, conjugate, delete, sum
from numpy.random import randint
from numpy.linalg import eigh

import matplotlib.pyplot as plt
import matplotlib as mpl

import json
from datetime import datetime

###API

In [ ]:
# Mudo o <IBM TOKEN> pelo IBM token (disponível em seu perfil da plataforma)
provider = IBMProvider(token='<IBM TOKEN>')
backends = provider.backends()

#Código

In [4]:
def criar_EPR(mensagem,qbit1=0, qbit2=1):
  for i in range(len(mensagem)):
    mensagem[i].h(0)
    mensagem[i].x(1)
    mensagem[i].cx(0,1)
    mensagem[i].z(1)

In [9]:
def obter_rotacoes(vetores):
  Px=Operator(Pauli('Z'))
  Py=Operator(Pauli('X'))
  Pz=Operator(Pauli('Y'))

  rotacoes=[]

  for i in range(len(vetores)):
    ket0=[1,0]
    ket1=[0,1]

    theta=vetores[i][0]
    phi=vetores[i][1]

    sigma=sin(theta)*cos(phi)*Px+sin(phi)*sin(theta)*Py+cos(theta)*Pz
    autovalor,autovetor=eigh(sigma)
    v0=autovetor[:,1]
    v1=autovetor[:,0]
    brav0=conjugate(v0).transpose()
    brav1=conjugate(v1).transpose()

    ket0=[1,0]
    ket1=[0,1]

    operador= outer(ket0,brav0) +outer(ket1,brav1)
    rotacoes.append(operador)


  return rotacoes

In [6]:
def medir_qbit(mensagem,escolhas, operadores, qbit, cbit, text='Op'):
  for i in range(len(mensagem)):
    rotacao=operadores[escolhas[i]]
    mensagem[i].barrier()
    mensagem[i].unitary(rotacao,[qbit], label='$'+text+str(escolhas[i])+'$')
    mensagem[i].measure(qbit,cbit)

In [10]:
for k in range(5):
    # Determinação das diferentes direções que alice projetará o spin
    a1=[pi/2,0]
    a2=[pi/2,pi/4]
    a3=[pi/2,pi/2]
    vetores_alice=[a1,a2,a3]
    operador_alice=obter_rotacoes(vetores_alice)

    # Determinação das diferentes direções que alice projetará o spin
    b1=[pi/2,pi/4]
    b2=[pi/2,pi/2]
    b3=[pi/2,3*pi/4]
    vetores_bob=[b1,b2,b3]
    operador_bob=obter_rotacoes(vetores_bob)


    # ------------------------Etapa 1---------------------------------
    n=9
    mensagem=[]
    for i in range(n):
      qc=QuantumCircuit(2,2)
      mensagem.append(qc)

    criar_EPR(mensagem) #Criação dos n pares emaranhados

    # ------------------------Etapa 2---------------------------------
    escolhas_alice=[0,1,2,   0,1,2,   0,1,2]
    escolhas_bob=  [0,1,2,   1,2,0,   2,0,1]

    medir_qbit(mensagem, escolhas_alice, operador_alice, qbit=0,cbit=0,text='A(a_') #Alcie realiza a projeção
    medir_qbit(mensagem, escolhas_bob, operador_bob, qbit=1,cbit=1,text='B(b_') #Bob realiza a projeção

    mensagem=transpile(mensagem)

    # Escolha um dos backends tirando o comentário de uma das linhas abaixo
    # backend = provider.get_backend('ibm_kyoto')
    # backend = provider.get_backend('ibm_brisbane')
    backend = Aer.get_backend('qasm_simulator')

    shots=2000
    result = execute(mensagem, backend, shots=shots).result()

    # Como geralmente o tempo de fila é longo, aconselho baixar os dados diretamente da sessão jobs da plataforma IBM Quantum

    counts = result.get_counts()



In [14]:
#Imprime os valores de S para ibm_brisbane
for i in range(1,6):
    caminho_arquivo_json = f'dados brisbane/brisbane_{i}.txt'

    # Carregue os dados do arquivo JSON
    with open(caminho_arquivo_json, 'r') as arquivo_json:
        dados_json = json.load(arquivo_json)

    # Crie um objeto Result do Qiskit usando os dados carregados
    resultado_qiskit = Result.from_dict(dados_json)

    # Obtenha as contagens usando get_counts()
    counts = resultado_qiskit.get_counts()

    # Exiba as contagens
    E11=(counts[0]['00']+counts[0]['11']-counts[0]['10']-counts[0]['01'])/shots
    E33=(counts[2]['00']+counts[2]['11']-counts[2]['10']-counts[2]['01'])/shots
    E31=(counts[5]['00']+counts[5]['11']-counts[5]['10']-counts[5]['01'])/shots
    E13=(counts[6]['00']+counts[6]['11']-counts[6]['10']-counts[6]['01'])/shots


    S=E11-E13+E31+E33
    print(S)


-2.099
-1.9080000000000001
-1.9000000000000001
-1.967
-1.979


In [15]:
#Imprime os valores de S para ibm_kyoto
for i in range(1,6):
    caminho_arquivo_json = f'dados kyoto/kyoto_{i}.txt'

    # Carregue os dados do arquivo JSON
    with open(caminho_arquivo_json, 'r') as arquivo_json:
        dados_json = json.load(arquivo_json)

    # Crie um objeto Result do Qiskit usando os dados carregados
    resultado_qiskit = Result.from_dict(dados_json)

    # Obtenha as contagens usando get_counts()
    counts = resultado_qiskit.get_counts()

    # Exiba as contagens
    E11=(counts[0]['00']+counts[0]['11']-counts[0]['10']-counts[0]['01'])/shots
    E33=(counts[2]['00']+counts[2]['11']-counts[2]['10']-counts[2]['01'])/shots
    E31=(counts[5]['00']+counts[5]['11']-counts[5]['10']-counts[5]['01'])/shots
    E13=(counts[6]['00']+counts[6]['11']-counts[6]['10']-counts[6]['01'])/shots


    S=E11-E13+E31+E33
    print(S)

-1.133
-1.175
-1.157
-1.229
-1.1219999999999999
